# Assignment 2 - DV2599 Maskininlärning
Victor Arvidsson

viar19@student.bth.se

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
import time
pd.options.mode.chained_assignment = None  # disable warning

## Load and analyze dataset

In [2]:
df = pd.read_csv('spambase.data')
df.head()

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total,spam
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1


In [3]:
df.describe()

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total,spam
count,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,...,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000
mean,0.104553,0.213015,0.280656,0.065425,0.312223,0.095901,0.114208,0.105295,0.090067,0.239413,...,0.038575,0.139030,0.016976,0.269071,0.075811,0.044238,5.191515,52.172789,283.289285,0.394045
std,0.305358,1.290575,0.504143,1.395151,0.672513,0.273824,0.391441,0.401071,0.278616,0.644755,...,0.243471,0.270355,0.109394,0.815672,0.245882,0.429342,31.729449,194.891310,606.347851,0.488698
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.588000,6.000000,35.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.065000,0.000000,0.000000,0.000000,0.000000,2.276000,15.000000,95.000000,0.000000
75%,0.000000,0.000000,0.420000,0.000000,0.380000,0.000000,0.000000,0.000000,0.000000,0.160000,...,0.000000,0.188000,0.000000,0.315000,0.052000,0.000000,3.706000,43.000000,266.000000,1.000000
max,4.540000,14.280000,5.100000,42.810000,10.000000,5.880000,7.270000,11.110000,5.260000,18.180000,...,4.385000,9.752000,4.081000,32.478000,6.003000,19.829000,1102.500000,9989.000000,15841.000000,1.000000


In [4]:
df_y = df['spam']
df_x = df.drop(columns=['spam'])

In [5]:
def evaluate(classifierCls, df_x, df_y):
    skf = StratifiedKFold(n_splits=10)
    step = 0
    for train_index, test_index in skf.split(df_x, df_y):
        x_train, x_test = df_x.iloc[train_index], df_x.iloc[test_index]
        y_train, y_test = df_y.iloc[train_index], df_y.iloc[test_index]
        classifier = classifierCls()
        t1 = time.perf_counter_ns()
        classifier = classifier.fit(x_train, y_train)
        predictions = classifier.predict(x_test)
        t2 = time.perf_counter_ns()
        FP = FN = TP = TN = 0
        for i in range(len(y_test)):
            real = y_test.iloc[i]
            predicted = predictions[i]
            if predicted == 1 and real == 1:
                TP += 1
            elif predicted == 1 and real == 0:
                FP += 1
            elif predicted == 0 and real == 1:
                FN += 1
            else:
                TN += 1
        accuracy = (TP + TN) / (TP + FP + FN + TN)
        precision = TP / (TP + FP)
        recall = TP / (TP + FN)
        f1 = 2 * precision * recall / (precision + recall)
        print(f'Step: {step}, Time performance: {(t2 - t1)/10**6:.1f} ms, Accuracy: {accuracy:.3f}, f1-score: {f1:.3f}')
        step += 1

# Classifier 1: K-Nearest neighbor classifier

In [6]:
evaluate(KNeighborsClassifier, df_x, df_y)


Step: 0, Time performance: 169.8 ms, Accuracy: 0.740, f1-score: 0.653
Step: 1, Time performance: 48.6 ms, Accuracy: 0.776, f1-score: 0.713
Step: 2, Time performance: 87.4 ms, Accuracy: 0.778, f1-score: 0.707
Step: 3, Time performance: 57.2 ms, Accuracy: 0.822, f1-score: 0.757
Step: 4, Time performance: 44.1 ms, Accuracy: 0.811, f1-score: 0.760
Step: 5, Time performance: 44.9 ms, Accuracy: 0.822, f1-score: 0.777
Step: 6, Time performance: 44.0 ms, Accuracy: 0.820, f1-score: 0.746
Step: 7, Time performance: 46.7 ms, Accuracy: 0.828, f1-score: 0.780
Step: 8, Time performance: 44.5 ms, Accuracy: 0.724, f1-score: 0.672
Step: 9, Time performance: 45.2 ms, Accuracy: 0.763, f1-score: 0.703


# Classifier 2: Gaussian Naive Bayesian

In [7]:
evaluate(GaussianNB, df_x, df_y)

Step: 0, Time performance: 13.3 ms, Accuracy: 0.844, f1-score: 0.829
Step: 1, Time performance: 12.8 ms, Accuracy: 0.863, f1-score: 0.850
Step: 2, Time performance: 12.1 ms, Accuracy: 0.878, f1-score: 0.864
Step: 3, Time performance: 11.6 ms, Accuracy: 0.867, f1-score: 0.854
Step: 4, Time performance: 12.7 ms, Accuracy: 0.885, f1-score: 0.866
Step: 5, Time performance: 11.1 ms, Accuracy: 0.828, f1-score: 0.819
Step: 6, Time performance: 11.7 ms, Accuracy: 0.833, f1-score: 0.821
Step: 7, Time performance: 11.6 ms, Accuracy: 0.867, f1-score: 0.848
Step: 8, Time performance: 11.7 ms, Accuracy: 0.635, f1-score: 0.669
Step: 9, Time performance: 11.0 ms, Accuracy: 0.717, f1-score: 0.711


# Classifier 3: Random forest

In [8]:
evaluate(RandomForestClassifier, df_x, df_y)

Step: 0, Time performance: 794.9 ms, Accuracy: 0.952, f1-score: 0.938
Step: 1, Time performance: 776.6 ms, Accuracy: 0.952, f1-score: 0.939
Step: 2, Time performance: 950.6 ms, Accuracy: 0.935, f1-score: 0.913
Step: 3, Time performance: 790.4 ms, Accuracy: 0.954, f1-score: 0.940
Step: 4, Time performance: 649.5 ms, Accuracy: 0.959, f1-score: 0.947
Step: 5, Time performance: 784.9 ms, Accuracy: 0.959, f1-score: 0.948
Step: 6, Time performance: 875.2 ms, Accuracy: 0.967, f1-score: 0.958
Step: 7, Time performance: 963.1 ms, Accuracy: 0.972, f1-score: 0.964
Step: 8, Time performance: 943.9 ms, Accuracy: 0.891, f1-score: 0.869
Step: 9, Time performance: 1063.5 ms, Accuracy: 0.857, f1-score: 0.816
